# 라이브러리

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import datetime
import re
import joblib

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from kmodes.kprototypes import KPrototypes

# 데이터 불러오기

In [ ]:
def call_df(table_name):
    with open('config.json', 'r') as f:
        config = json.load(f)
        
    conn = psycopg2.connect(user = config['USER'],
                              password = config['PASSWORD'],
                              host = config['HOST'],
                              port = config['PORT'],
                              database = config['DATABASE'])
    
    sql = f'SELECT * FROM {table_name}'
    df = pd.read_sql_query(sql, conn)
    conn.close()
    return df

In [ ]:
district = call_df('crawling_db.district_table')
apartment = call_df('crawling_db.apartment_table').drop(columns='table_id')
school = call_df('crawling_db.school_table').drop(columns='table_id')
subway = call_df('crawling_db.subway_table').drop(columns='table_id')
price = call_df('crawling_db.price_table')

In [ ]:
df = (apartment.
      merge(district, how='left', on='district_id').
      merge(school, how='left', on='apartment_id').
      merge(subway, how='left',on='apartment_id'))

# 데이터 전처리

## 숫자형 자료

In [ ]:
df['apartment_floor_min'] = pd.to_numeric(df['apartment_floor_min'])
df['apartment_floor_max'] = pd.to_numeric(df['apartment_floor_max'])
df['school_students'] = pd.to_numeric(df['school_students'])
df['school_addr_town'] = df['school_addr_town'].apply(lambda x: re.split('\d',x)[0])

In [ ]:
print(df.isna().sum())

- 네이버에서 근처에 학교가 없을 경우, 학교가 없다고 인식하여 결측치가 발생합니다. 이를 0으로 대체하겠습니다.

In [ ]:
df['school_students'] = df['school_students'].fillna(0)

### 정규화
- 표준정규분포 정규화

In [ ]:
from sklearn.preprocessing import StandardScaler

def standard_scaler(data, variable):
    scaler = StandardScaler()
    scaler.fit(data[variable].values.reshape(-1,1))
    joblib.dump(scaler, f'./model/{variable}_scaler.pkl')

def scaling(data, variable):
    scaler = joblib.load(f'./model/{variable}_scaler.pkl')
    output = scaler.transform(data[variable].values.reshape(-1,1))
    return output

def inverse_scaling(data, variable):
    scaler = joblib.load(f'./model/{variable}_scaler.pkl')
    output = scaler.inverse_transform(data[variable].values.reshape(-1,1))
    return output 

In [ ]:
numericals = [
    'apartment_floor_min',
    'apartment_floor_max',
    'apartment_parking',
    'school_students',
    'st_dist',
    'st_volume'
    ]

for variable in numericals:
    standard_scaler(df, variable)
    df[variable] = scaling(df, variable)

## 범주형 자료

In [ ]:
labels = [
    'apartment_addr_town',
    'apartment_builder',
    'school_name',
    'school_addr_town',
    'st_name'
    ]

categoricals = [
    'district_id',
    'apartment_addr_town',
    'apartment_builder',
    'apartment_build_year',
    'apartment_build_month',
    'school_name',
    'school_addr_district',
    'school_addr_town',
    'st_name'
    ]

In [ ]:
def label_encoding(data, variable):
    encoder = joblib.load(f'./model/{variable}_encoder.pkl')
    output = encoder.transform(data[variable])
    return output

def label_decoding(data, variable):
    encoder = joblib.load(f'./model/{variable}_encoder.pkl')
    output = encoder.inverse_transform(data[variable])
    return output

In [ ]:
for variable in labels:
    df[variable] = label_encoding(df, variable)

# Modeling 

## K-Prototypes Clustering

In [ ]:
train = df[categoricals + numericals].copy()
print(len(categoricals))

In [ ]:
train.info()

In [ ]:
n_pseudo = range(1,21) 
cost = []
for i in tqdm(n_pseudo):
    kproto = KPrototypes(n_clusters=i, verbose=0, random_state=0, n_jobs=-1)
    kproto.fit_predict(train, categorical=list(range(len(categoricals))))
    cost.append(kproto.cost_)
cost_table = pd.DataFrame({'n':n_pseudo, 'cost':cost})

In [ ]:
def cost_visualize(data, x, y):
    sns.pointplot(data=data, x=x, y=y)
    plt.title('Cost Visualization')
    plt.show()
cost_visualize(cost_table, 'n', 'cost')

- 적당한 k를 찾기 위해 Elbow Method를 사용합니다. Cost가 줄어드는 비율이 확연하게 줄어드는 지점이 유의미한 k라고 가정합니다. 위의 Cost 시각화 그래프에서 기울기의 절대값이 변동하는 비율이 0.05 이하로 떨어지는 시점을 적당한 k라고 정의하겠습니다.   

In [ ]:
cost_table['coef'] = abs(cost_table['cost'] - cost_table['cost'].shift(1))
cost_table['ratio'] =  abs((cost_table['coef'].shift(1) - cost_table['coef'])/cost_table['coef'])

In [ ]:
cost_table

In [ ]:
kproto = KPrototypes(n_clusters=8, verbose=0, random_state=0, n_jobs=-1)
train['cluster'] = kproto.fit_predict(train, categorical=list(range(len(categoricals))))
joblib.dump(kproto, './model/kprototype_model.pkl')

## Decoding Into Original

In [ ]:
train['apartment_id'] = df['apartment_id']
train['aparmtent_name'] = df['apartment_name']
train['district_name'] = df['district_name']

for variable in numericals:
    train[variable] = inverse_scaling(train, variable)
for variable in labels:
    train[variable] = label_decoding(train, variable)  

# Visualization